In [1]:
import time
import pandas as pd
import numpy as np
import math
from math import cos, asin, sqrt
import gc
pd.set_option('display.max_columns', None)


#ejecutar esto solo la primera vez#####################################################
start = time.time()
vuelos = pd.read_csv('vuelosDeparted.csv', sep=',',low_memory=False, )
vuelos["TMIN_o"]= ''
vuelos["TMIN_d"]= ''
vuelos["TMAX_o"]= ''
vuelos["TMAX_d"]= ''
vuelos["TAVG_o"]= ''
vuelos["TAVG_d"]= ''
vuelos["SNOW_o"]= ''
vuelos["SNOW_d"]= ''
vuelos["PRCP_o"]= ''
vuelos["PRCP_d"]= ''
vuelos["SNWD_o"]= ''
vuelos["SNWD_d"]= ''
vuelos["ACMC_o"]= ''
vuelos["ACMC_d"]= ''
vuelos["ACSC_o"]= ''
vuelos["ACSC_d"]= ''
vuelos["AWDR_o"]= ''
vuelos["AWDR_d"]= ''
vuelos["AWND_o"]= ''
vuelos["AWND_d"]= ''
vuelos["EVAP_o"]= ''
vuelos["EVAP_d"]= ''
vuelos["FRTH_o"]= ''
vuelos["FRTH_d"]= ''
vuelos["TSUN_o"]= ''
vuelos["TSUN_d"]= ''
vuelos["WDMV_o"]= ''
vuelos["WDMV_d"]= ''



vuelos.to_csv('vuelos.csv', sep=',', index=False)
vuelos = pd.read_csv('vuelos.csv', sep=',',low_memory=False, )
vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
end = time.time()
print(end - start)

In [ ]:
!wget http://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt -O stations.txt


In [2]:
vuelos = pd.read_csv('vuelos.csv', sep=',',low_memory=False, )
vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
# Load of geobase
start = time.time()
from GeoBases import GeoBase
geo_c = GeoBase(data='ori_por', verbose=False)
stationstxt = ""
with open("stations.txt") as input:
    stationstxt = input.read()
    
#Extract the data from file
stations2 = stationstxt.split("\n")
#Remove last line
stations2 = stations2[:-1]
stations2 = map(lambda line: [line[0:11],float(line[13:20]),float(line[22:30]),line[41:71]], stations2)

#weather2012 = pd.read_csv("2012.csv",header=None)
#weather2012.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]

#weather2011 = pd.read_csv("2011.csv",header=None)
#weather2011.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2014 = pd.read_csv("2014.csv",header=None)
#weather2014.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2015 = pd.read_csv("2015.csv",header=None)
#weather2015.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
#weather2016 = pd.read_csv("2016.csv",header=None)
#weather2016.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
weather2016 = pd.read_csv("2016.csv",header=None)
weather2016.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]
end = time.time()
print(end - start)

35.993999958


del weather2015
#del weather2014
#del weather2012
gc.collect()

In [3]:
stationsdf = pd.DataFrame(stations2)
stationsdf.columns = ["id","lat","lng","name"]
lats = stationsdf[['lat','lng']]

In [4]:
#funciones para calcular la distancia entre los aeropuertos y estacion metereologica mas cercana
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v[0],v[1],p[0],p[1]))

#para que la fecha no de problemas al comparar (quizas no este bien del todo)
def tratarFecha(fecha):
    fecha = str(fecha).split(' ')[0]
    fecha = fecha.replace('-','')
    fecha = int(fecha)
    return fecha

In [5]:
#para seleccionar con que año trabajar
weatherdf = weather2016

In [6]:
del weatherdf['Value3']
del weatherdf['Value4']
del weatherdf['Value5']
del weatherdf['Value2']

In [7]:
def tratar2(row):
    lat = row['board_lat']
    lon = row['board_lon']
    fecha = row['actual_time_of_departure']
    lat_d = row['off_lat']
    lon_d = row['off_lon']
    fecha_d = row['actual_time_of_arrival'] 
    ##################################
    #primero tratamos las fechas
    fecha = tratarFecha(fecha)
    fecha_d = tratarFecha(fecha_d)
    ##################################
    coor = [lat,lon]
    coor_off = [lat_d,lon_d]
    a= closest(np.asarray(lats), coor)
    b= closest(np.asarray(lats), coor_off)
    ##################################
    aux = stationsdf[stationsdf['lat']==a[0]]
    aux = stationsdf[stationsdf['lng']==a[1]]
    stationid = aux['id']
    s =stationid.to_string()
    stationid = str(s)
    stationid = stationid.split()[1]
    ##################################
    aux1 = stationsdf[stationsdf['lat']==b[0]]
    aux1 = stationsdf[stationsdf['lng']==b[1]]
    stationidoff = aux1['id']
    s =stationidoff.to_string()
    stationidoff = str(s)
    stationidoff = stationidoff.split()[1]
    #########################################################################################################
    data_board = weatherdf[(weatherdf["id"]==stationid)&(weatherdf['date']==fecha)].sort_values("date")
    data_off   = weatherdf[(weatherdf["id"]==stationidoff)&(weatherdf['date']==fecha_d)].sort_values("date")
    #####################################################
    data_board["Value1"]=data_board["Value1"]/10
    data_board = data_board.reset_index()
    data_off["Value1"]=data_off["Value1"]/10
    data_off = data_off.reset_index()
        #######################################################
    T1 = data_board[data_board['type']=='TMIN']['Value1']
    T2 = data_off[data_off['type']=='TMIN']['Value1']
    T3 = data_board[data_board['type']=='TMAX']['Value1']
    T4 = data_off[data_off['type']=='TMAX']['Value1']
    T5 = data_board[data_board['type']=='TAVG']['Value1']
    T6 = data_off[data_off['type']=='TAVG']['Value1']
    T7 = data_board[data_board['type']=='SNOW']['Value1']
    T8 = data_off[data_off['type']=='SNOW']['Value1']
    T9 = data_board[data_board['type']=='PRCP']['Value1']
    T10 = data_off[data_off['type']=='PRCP']['Value1']
    T11 = data_board[data_board['type']=='SNWD']['Value1']
    T12 = data_off[data_off['type']=='SNWD']['Value1']
    T13 = data_board[data_board['type']=='ACMC']['Value1']
    T14 = data_off[data_off['type']=='ACMC']['Value1']
    T17 = data_board[data_board['type']=='ACSC']['Value1']
    T18 = data_off[data_off['type']=='ACSC']['Value1']
    T21 = data_board[data_board['type']=='AWDR']['Value1']
    T22 = data_off[data_off['type']=='AWDR']['Value1']
    T23 = data_board[data_board['type']=='AWND']['Value1']
    T24 = data_off[data_off['type']=='AWND']['Value1']
    T39 = data_board[data_board['type']=='EVAP']['Value1']
    T40 = data_off[data_off['type']=='EVAP']['Value1']
    T47 = data_board[data_board['type']=='FRTH']['Value1']
    T48 = data_off[data_off['type']=='FRTH']['Value1']
    T75 = data_board[data_board['type']=='TSUN']['Value1']
    T76 = data_off[data_off['type']=='TSUN']['Value1']
    T89 = data_board[data_board['type']=='WDMV']['Value1']
    T90 = data_off[data_off['type']=='WDMV']['Value1']
    
    
    ##############################
    ##tratamos los valores nulos aplicando un valor nulo
    if T1.empty:
        T1 = 9999
    if T2.empty:
        T2 = 9999
    if T3.empty:
        T3 = 9999
    if T4.empty:
        T4 = 9999
    if T5.empty:
        T5 = 9999
    if T6.empty:
        T6 = 9999
    if T7.empty:
        T7 = 9999
    if T8.empty:
        T8 = 9999
    if T9.empty:
        T9 = 9999
    if T10.empty:
        T10 = 9999
    if T11.empty:
        T11 = 9999
    if T12.empty:
        T12 = 9999
    if T13.empty:
        T13 = 9999
    if T14.empty:
        T14 = 9999
    if T17.empty:
        T17 = 9999
    if T18.empty:
        T18 = 9999
    if T21.empty:
        T21 = 9999
    if T22.empty:
        T22 = 9999
    if T23.empty:
        T23 = 9999
    if T24.empty:
        T24 = 9999
    if T39.empty:
        T39 = 9999
    if T40.empty:
        T40 = 9999
    if T47.empty:
        T47 = 9999
    if T48.empty:
        T48 = 9999
    if T75.empty:
        T75 = 9999
    if T76.empty:
        T76 = 9999
    if T89.empty:
        T89 = 9999
    if T90.empty:
        T90 = 9999
      
        
    #actualizamos la fila
    row['TMIN_o']= float(T1)
    row['TMIN_d']= float(T2)
    row['TMAX_o']= float(T3)
    row['TMAX_d']= float(T4)
    row['TAVG_o']= float(T5)
    row['TAVG_d']= float(T6)
    row['SNOW_o']= float(T7)
    row['SNOW_d']= float(T8)
    row['PRCP_o']= float(T9)
    row['PRCP_d']= float(T10)
    row['SNWD_o']= float(T11)
    row['SNWD_d']= float(T12)
    row['ACMC_o']= float(T13)
    row['ACMC_d']= float(T14)
    row['ACSC_o']= float(T17)
    row['ACSC_d']= float(T18)
    row['AWDR_o']= float(T21)
    row['AWDR_d']= float(T22)
    row['AWND_o']= float(T23)
    row['AWND_d']= float(T24)
    row['EVAP_o']= float(T39)
    row['EVAP_d']= float(T40)
    row['FRTH_o']= float(T47)
    row['FRTH_d']= float(T48)
    row['TSUN_o']= float(T75)
    row['TSUN_d']= float(T76)
    row['WDMV_o']= float(T89)
    row['WDMV_d']= float(T90)


    return row

In [8]:
#para saber que vuelos tenemos de que años
vuelos2017 = vuelos[vuelos['anyoSalida']==2017]
vuelos2016 = vuelos[vuelos['anyoSalida']==2016]
vuelos2015 = vuelos[vuelos['anyoSalida']==2015]
vuelos2014 = vuelos[vuelos['anyoSalida']==2014]
vuelos2013 = vuelos[vuelos['anyoSalida']==2013]
vuelos2012 = vuelos[vuelos['anyoSalida']==2012]
vuelos2011 = vuelos[vuelos['anyoSalida']==2011]
vuelos2010 = vuelos[vuelos['anyoSalida']==2010]
vuelos2009 = vuelos[vuelos['anyoSalida']==2009]

#ver las dimensiones de cada año
print vuelos2017.shape
print vuelos2016.shape
print vuelos2015.shape
print vuelos2014.shape
print vuelos2013.shape
print vuelos2012.shape
print vuelos2011.shape
print vuelos2010.shape
print vuelos2009.shape

(24044, 162)
(137910, 162)
(33894, 162)
(466, 162)
(0, 162)
(3, 162)
(0, 162)
(0, 162)
(0, 162)


In [9]:
vuelos = pd.read_csv('vuelos.csv', sep=',',low_memory=False, )
vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
vuelosTratados = vuelos[vuelos['TAVG_o'].notnull()]
print vuelosSinTratar.shape
print vuelosTratados.shape
x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==2016]
print x.shape


(141005, 162)
(55312, 162)
(128610, 162)


In [10]:
print x['mesSalida'].unique()

[ 7  8  9 10 11 12  6  5  4  2  3  1]


In [11]:
def functionWeatherDay(month, day):
    u = int(month+day)
    weatherdf = weather2016
    weatherdf = weatherdf[weatherdf['date']-20160000==u]
    return weatherdf

In [12]:
def functionPlus(year, month, day):    
    start = time.time()
    #vuelos = pd.read_csv('vuelos.csv', sep=',',low_memory=False, )
    #vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
    #nos quedamos con los vuelos de 2012
    x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==year]
    x = x[x['mesSalida']==month]
    x = x[x['diaSalida']==day]
    ##cogemos un rango para trabajar con el
    x = x.head(50)################################################################Nº A PROCESAR
    x = x.apply(lambda x:tratar2(x),axis = 1)
    #para escribir ya
    #variable auxiliar

    pd.options.mode.chained_assignment = None  # default='warn'
    vuelosSinTratar['TMIN_o'] = x.set_index(['index'])['TMIN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_o']).values
    vuelosSinTratar['TMIN_d'] = x.set_index(['index'])['TMIN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_d']).values
    vuelosSinTratar['TMAX_o'] = x.set_index(['index'])['TMAX_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_o']).values
    vuelosSinTratar['TMAX_d'] = x.set_index(['index'])['TMAX_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_d']).values
    vuelosSinTratar['TAVG_o'] = x.set_index(['index'])['TAVG_o'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_o']).values
    vuelosSinTratar['TAVG_d'] = x.set_index(['index'])['TAVG_d'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_d']).values
    vuelosSinTratar['SNOW_o'] = x.set_index(['index'])['SNOW_o'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_o']).values
    vuelosSinTratar['SNOW_d'] = x.set_index(['index'])['SNOW_d'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_d']).values
    vuelosSinTratar['PRCP_o'] = x.set_index(['index'])['PRCP_o'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_o']).values
    vuelosSinTratar['PRCP_d'] = x.set_index(['index'])['PRCP_d'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_d']).values
    vuelosSinTratar['SNWD_o'] = x.set_index(['index'])['SNWD_o'].combine_first(vuelosSinTratar.set_index(['index'])['SNWD_o']).values
    vuelosSinTratar['SNWD_d'] = x.set_index(['index'])['SNWD_d'].combine_first(vuelosSinTratar.set_index(['index'])['SNWD_d']).values
    vuelosSinTratar['ACMC_o'] = x.set_index(['index'])['ACMC_o'].combine_first(vuelosSinTratar.set_index(['index'])['ACMC_o']).values
    vuelosSinTratar['ACMC_d'] = x.set_index(['index'])['ACMC_d'].combine_first(vuelosSinTratar.set_index(['index'])['ACMC_d']).values
    vuelosSinTratar['ACSC_o'] = x.set_index(['index'])['ACSC_o'].combine_first(vuelosSinTratar.set_index(['index'])['ACSC_o']).values
    vuelosSinTratar['ACSC_d'] = x.set_index(['index'])['ACSC_d'].combine_first(vuelosSinTratar.set_index(['index'])['ACSC_d']).values
    vuelosSinTratar['AWDR_o'] = x.set_index(['index'])['AWDR_o'].combine_first(vuelosSinTratar.set_index(['index'])['AWDR_o']).values
    vuelosSinTratar['AWDR_d'] = x.set_index(['index'])['AWDR_d'].combine_first(vuelosSinTratar.set_index(['index'])['AWDR_d']).values
    vuelosSinTratar['AWND_o'] = x.set_index(['index'])['AWND_o'].combine_first(vuelosSinTratar.set_index(['index'])['AWND_o']).values
    vuelosSinTratar['AWND_d'] = x.set_index(['index'])['AWND_d'].combine_first(vuelosSinTratar.set_index(['index'])['AWND_d']).values
    vuelosSinTratar['EVAP_o'] = x.set_index(['index'])['EVAP_o'].combine_first(vuelosSinTratar.set_index(['index'])['EVAP_o']).values
    vuelosSinTratar['EVAP_d'] = x.set_index(['index'])['EVAP_d'].combine_first(vuelosSinTratar.set_index(['index'])['EVAP_d']).values
    vuelosSinTratar['FRTH_o'] = x.set_index(['index'])['FRTH_o'].combine_first(vuelosSinTratar.set_index(['index'])['FRTH_o']).values
    vuelosSinTratar['FRTH_d'] = x.set_index(['index'])['FRTH_d'].combine_first(vuelosSinTratar.set_index(['index'])['FRTH_d']).values
    vuelosSinTratar['TSUN_o'] = x.set_index(['index'])['TSUN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TSUN_o']).values
    vuelosSinTratar['TSUN_d'] = x.set_index(['index'])['TSUN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TSUN_d']).values
    vuelosSinTratar['WDMV_o'] = x.set_index(['index'])['WDMV_o'].combine_first(vuelosSinTratar.set_index(['index'])['WDMV_o']).values
    vuelosSinTratar['WDMV_d'] = x.set_index(['index'])['WDMV_d'].combine_first(vuelosSinTratar.set_index(['index'])['WDMV_d']).values


    vuelos['TMIN_o'] = vuelosSinTratar.set_index(['index'])['TMIN_o'].combine_first(vuelos.set_index(['index'])['TMIN_o']).values
    vuelos['TMIN_d'] = vuelosSinTratar.set_index(['index'])['TMIN_d'].combine_first(vuelos.set_index(['index'])['TMIN_d']).values
    vuelos['TMAX_o'] = vuelosSinTratar.set_index(['index'])['TMAX_o'].combine_first(vuelos.set_index(['index'])['TMAX_o']).values
    vuelos['TMAX_d'] = vuelosSinTratar.set_index(['index'])['TMAX_d'].combine_first(vuelos.set_index(['index'])['TMAX_d']).values
    vuelos['TAVG_o'] = vuelosSinTratar.set_index(['index'])['TAVG_o'].combine_first(vuelos.set_index(['index'])['TAVG_o']).values
    vuelos['TAVG_d'] = vuelosSinTratar.set_index(['index'])['TAVG_d'].combine_first(vuelos.set_index(['index'])['TAVG_d']).values
    vuelos['SNOW_o'] = vuelosSinTratar.set_index(['index'])['SNOW_o'].combine_first(vuelos.set_index(['index'])['SNOW_o']).values
    vuelos['SNOW_d'] = vuelosSinTratar.set_index(['index'])['SNOW_d'].combine_first(vuelos.set_index(['index'])['SNOW_d']).values
    vuelos['PRCP_o'] = vuelosSinTratar.set_index(['index'])['PRCP_o'].combine_first(vuelos.set_index(['index'])['PRCP_o']).values
    vuelos['PRCP_d'] = vuelosSinTratar.set_index(['index'])['PRCP_d'].combine_first(vuelos.set_index(['index'])['PRCP_d']).values
    vuelos['SNWD_o'] = vuelosSinTratar.set_index(['index'])['SNWD_o'].combine_first(vuelos.set_index(['index'])['SNWD_o']).values
    vuelos['SNWD_d'] = vuelosSinTratar.set_index(['index'])['SNWD_d'].combine_first(vuelos.set_index(['index'])['SNWD_d']).values
    vuelos['ACMC_o'] = vuelosSinTratar.set_index(['index'])['ACMC_o'].combine_first(vuelos.set_index(['index'])['ACMC_o']).values
    vuelos['ACMC_d'] = vuelosSinTratar.set_index(['index'])['ACMC_d'].combine_first(vuelos.set_index(['index'])['ACMC_d']).values
    vuelos['ACSC_o'] = vuelosSinTratar.set_index(['index'])['ACSC_o'].combine_first(vuelos.set_index(['index'])['ACSC_o']).values
    vuelos['ACSC_d'] = vuelosSinTratar.set_index(['index'])['ACSC_d'].combine_first(vuelos.set_index(['index'])['ACSC_d']).values
    vuelos['AWDR_o'] = vuelosSinTratar.set_index(['index'])['AWDR_o'].combine_first(vuelos.set_index(['index'])['AWDR_o']).values
    vuelos['AWDR_d'] = vuelosSinTratar.set_index(['index'])['AWDR_d'].combine_first(vuelos.set_index(['index'])['AWDR_d']).values
    vuelos['AWND_o'] = vuelosSinTratar.set_index(['index'])['AWND_o'].combine_first(vuelos.set_index(['index'])['AWND_o']).values
    vuelos['AWND_d'] = vuelosSinTratar.set_index(['index'])['AWND_d'].combine_first(vuelos.set_index(['index'])['AWND_d']).values
    vuelos['EVAP_o'] = vuelosSinTratar.set_index(['index'])['EVAP_o'].combine_first(vuelos.set_index(['index'])['EVAP_o']).values
    vuelos['EVAP_d'] = vuelosSinTratar.set_index(['index'])['EVAP_d'].combine_first(vuelos.set_index(['index'])['EVAP_d']).values
    vuelos['FRTH_o'] = vuelosSinTratar.set_index(['index'])['FRTH_o'].combine_first(vuelos.set_index(['index'])['FRTH_o']).values
    vuelos['FRTH_d'] = vuelosSinTratar.set_index(['index'])['FRTH_d'].combine_first(vuelos.set_index(['index'])['FRTH_d']).values
    vuelos['TSUN_o'] = vuelosSinTratar.set_index(['index'])['TSUN_o'].combine_first(vuelos.set_index(['index'])['TSUN_o']).values
    vuelos['TSUN_d'] = vuelosSinTratar.set_index(['index'])['TSUN_d'].combine_first(vuelos.set_index(['index'])['TSUN_d']).values
    vuelos['WDMV_o'] = vuelosSinTratar.set_index(['index'])['WDMV_o'].combine_first(vuelos.set_index(['index'])['WDMV_o']).values
    vuelos['WDMV_d'] = vuelosSinTratar.set_index(['index'])['WDMV_d'].combine_first(vuelos.set_index(['index'])['WDMV_d']).values

    
    #vuelos.to_csv('vuelos.csv', sep=',', index=False)
    end = time.time()
    print(end - start)

In [13]:
def functionPlus2(year, month, day):    
    start = time.time()
    #vuelos = pd.read_csv('vuelos.csv', sep=',',low_memory=False, )
    #vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
    #nos quedamos con los vuelos de 2012
    x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==year]
    x = x[x['mesSalida']==month]
    x = x[x['diaSalida']==day]
    ##cogemos un rango para trabajar con el
    x = x.head(50)################################################################Nº A PROCESAR
   # x = x.apply(lambda x:tratar2(x),axis = 1)
    x = x.map(lambda x:tratar2(x),axis = 1)
    #para escribir ya
    #variable auxiliar

    pd.options.mode.chained_assignment = None  # default='warn'
    vuelosSinTratar['TMIN_o'] = x.set_index(['index'])['TMIN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_o']).values
    vuelosSinTratar['TMIN_d'] = x.set_index(['index'])['TMIN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_d']).values
    vuelosSinTratar['TMAX_o'] = x.set_index(['index'])['TMAX_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_o']).values
    vuelosSinTratar['TMAX_d'] = x.set_index(['index'])['TMAX_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_d']).values
    vuelosSinTratar['TAVG_o'] = x.set_index(['index'])['TAVG_o'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_o']).values
    vuelosSinTratar['TAVG_d'] = x.set_index(['index'])['TAVG_d'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_d']).values
    vuelosSinTratar['SNOW_o'] = x.set_index(['index'])['SNOW_o'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_o']).values
    vuelosSinTratar['SNOW_d'] = x.set_index(['index'])['SNOW_d'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_d']).values
    vuelosSinTratar['PRCP_o'] = x.set_index(['index'])['PRCP_o'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_o']).values
    vuelosSinTratar['PRCP_d'] = x.set_index(['index'])['PRCP_d'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_d']).values
    vuelosSinTratar['SNWD_o'] = x.set_index(['index'])['SNWD_o'].combine_first(vuelosSinTratar.set_index(['index'])['SNWD_o']).values
    vuelosSinTratar['SNWD_d'] = x.set_index(['index'])['SNWD_d'].combine_first(vuelosSinTratar.set_index(['index'])['SNWD_d']).values
    vuelosSinTratar['ACMC_o'] = x.set_index(['index'])['ACMC_o'].combine_first(vuelosSinTratar.set_index(['index'])['ACMC_o']).values
    vuelosSinTratar['ACMC_d'] = x.set_index(['index'])['ACMC_d'].combine_first(vuelosSinTratar.set_index(['index'])['ACMC_d']).values
    vuelosSinTratar['ACSC_o'] = x.set_index(['index'])['ACSC_o'].combine_first(vuelosSinTratar.set_index(['index'])['ACSC_o']).values
    vuelosSinTratar['ACSC_d'] = x.set_index(['index'])['ACSC_d'].combine_first(vuelosSinTratar.set_index(['index'])['ACSC_d']).values
    vuelosSinTratar['AWDR_o'] = x.set_index(['index'])['AWDR_o'].combine_first(vuelosSinTratar.set_index(['index'])['AWDR_o']).values
    vuelosSinTratar['AWDR_d'] = x.set_index(['index'])['AWDR_d'].combine_first(vuelosSinTratar.set_index(['index'])['AWDR_d']).values
    vuelosSinTratar['AWND_o'] = x.set_index(['index'])['AWND_o'].combine_first(vuelosSinTratar.set_index(['index'])['AWND_o']).values
    vuelosSinTratar['AWND_d'] = x.set_index(['index'])['AWND_d'].combine_first(vuelosSinTratar.set_index(['index'])['AWND_d']).values
    vuelosSinTratar['EVAP_o'] = x.set_index(['index'])['EVAP_o'].combine_first(vuelosSinTratar.set_index(['index'])['EVAP_o']).values
    vuelosSinTratar['EVAP_d'] = x.set_index(['index'])['EVAP_d'].combine_first(vuelosSinTratar.set_index(['index'])['EVAP_d']).values
    vuelosSinTratar['FRTH_o'] = x.set_index(['index'])['FRTH_o'].combine_first(vuelosSinTratar.set_index(['index'])['FRTH_o']).values
    vuelosSinTratar['FRTH_d'] = x.set_index(['index'])['FRTH_d'].combine_first(vuelosSinTratar.set_index(['index'])['FRTH_d']).values
    vuelosSinTratar['TSUN_o'] = x.set_index(['index'])['TSUN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TSUN_o']).values
    vuelosSinTratar['TSUN_d'] = x.set_index(['index'])['TSUN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TSUN_d']).values
    vuelosSinTratar['WDMV_o'] = x.set_index(['index'])['WDMV_o'].combine_first(vuelosSinTratar.set_index(['index'])['WDMV_o']).values
    vuelosSinTratar['WDMV_d'] = x.set_index(['index'])['WDMV_d'].combine_first(vuelosSinTratar.set_index(['index'])['WDMV_d']).values


    vuelos['TMIN_o'] = vuelosSinTratar.set_index(['index'])['TMIN_o'].combine_first(vuelos.set_index(['index'])['TMIN_o']).values
    vuelos['TMIN_d'] = vuelosSinTratar.set_index(['index'])['TMIN_d'].combine_first(vuelos.set_index(['index'])['TMIN_d']).values
    vuelos['TMAX_o'] = vuelosSinTratar.set_index(['index'])['TMAX_o'].combine_first(vuelos.set_index(['index'])['TMAX_o']).values
    vuelos['TMAX_d'] = vuelosSinTratar.set_index(['index'])['TMAX_d'].combine_first(vuelos.set_index(['index'])['TMAX_d']).values
    vuelos['TAVG_o'] = vuelosSinTratar.set_index(['index'])['TAVG_o'].combine_first(vuelos.set_index(['index'])['TAVG_o']).values
    vuelos['TAVG_d'] = vuelosSinTratar.set_index(['index'])['TAVG_d'].combine_first(vuelos.set_index(['index'])['TAVG_d']).values
    vuelos['SNOW_o'] = vuelosSinTratar.set_index(['index'])['SNOW_o'].combine_first(vuelos.set_index(['index'])['SNOW_o']).values
    vuelos['SNOW_d'] = vuelosSinTratar.set_index(['index'])['SNOW_d'].combine_first(vuelos.set_index(['index'])['SNOW_d']).values
    vuelos['PRCP_o'] = vuelosSinTratar.set_index(['index'])['PRCP_o'].combine_first(vuelos.set_index(['index'])['PRCP_o']).values
    vuelos['PRCP_d'] = vuelosSinTratar.set_index(['index'])['PRCP_d'].combine_first(vuelos.set_index(['index'])['PRCP_d']).values
    vuelos['SNWD_o'] = vuelosSinTratar.set_index(['index'])['SNWD_o'].combine_first(vuelos.set_index(['index'])['SNWD_o']).values
    vuelos['SNWD_d'] = vuelosSinTratar.set_index(['index'])['SNWD_d'].combine_first(vuelos.set_index(['index'])['SNWD_d']).values
    vuelos['ACMC_o'] = vuelosSinTratar.set_index(['index'])['ACMC_o'].combine_first(vuelos.set_index(['index'])['ACMC_o']).values
    vuelos['ACMC_d'] = vuelosSinTratar.set_index(['index'])['ACMC_d'].combine_first(vuelos.set_index(['index'])['ACMC_d']).values
    vuelos['ACSC_o'] = vuelosSinTratar.set_index(['index'])['ACSC_o'].combine_first(vuelos.set_index(['index'])['ACSC_o']).values
    vuelos['ACSC_d'] = vuelosSinTratar.set_index(['index'])['ACSC_d'].combine_first(vuelos.set_index(['index'])['ACSC_d']).values
    vuelos['AWDR_o'] = vuelosSinTratar.set_index(['index'])['AWDR_o'].combine_first(vuelos.set_index(['index'])['AWDR_o']).values
    vuelos['AWDR_d'] = vuelosSinTratar.set_index(['index'])['AWDR_d'].combine_first(vuelos.set_index(['index'])['AWDR_d']).values
    vuelos['AWND_o'] = vuelosSinTratar.set_index(['index'])['AWND_o'].combine_first(vuelos.set_index(['index'])['AWND_o']).values
    vuelos['AWND_d'] = vuelosSinTratar.set_index(['index'])['AWND_d'].combine_first(vuelos.set_index(['index'])['AWND_d']).values
    vuelos['EVAP_o'] = vuelosSinTratar.set_index(['index'])['EVAP_o'].combine_first(vuelos.set_index(['index'])['EVAP_o']).values
    vuelos['EVAP_d'] = vuelosSinTratar.set_index(['index'])['EVAP_d'].combine_first(vuelos.set_index(['index'])['EVAP_d']).values
    vuelos['FRTH_o'] = vuelosSinTratar.set_index(['index'])['FRTH_o'].combine_first(vuelos.set_index(['index'])['FRTH_o']).values
    vuelos['FRTH_d'] = vuelosSinTratar.set_index(['index'])['FRTH_d'].combine_first(vuelos.set_index(['index'])['FRTH_d']).values
    vuelos['TSUN_o'] = vuelosSinTratar.set_index(['index'])['TSUN_o'].combine_first(vuelos.set_index(['index'])['TSUN_o']).values
    vuelos['TSUN_d'] = vuelosSinTratar.set_index(['index'])['TSUN_d'].combine_first(vuelos.set_index(['index'])['TSUN_d']).values
    vuelos['WDMV_o'] = vuelosSinTratar.set_index(['index'])['WDMV_o'].combine_first(vuelos.set_index(['index'])['WDMV_o']).values
    vuelos['WDMV_d'] = vuelosSinTratar.set_index(['index'])['WDMV_d'].combine_first(vuelos.set_index(['index'])['WDMV_d']).values

    
    #vuelos.to_csv('vuelos.csv', sep=',', index=False)
    end = time.time()
    print(end - start)

In [14]:
dia = 1
mes = 1
anyo = 2016

In [15]:
for mes in range(7,8):
    print 'empiezo el mes', mes
    mesw = str(mes)
    for dia in range(1,32):
        print 'empiezo el dia - ' ,dia
        diaw = str(dia)
        if dia < 10:
            diaw = '0'+diaw

        weatherdf = functionWeatherDay(mesw,diaw)
        functionPlus2(anyo,mes,dia)


empiezo el mes 7
empiezo el dia -  1


AttributeError: 'DataFrame' object has no attribute 'map'

In [ ]:
vuelos.to_csv('vuelos.csv', sep=',', index=False)

In [18]:
vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
vuelosTratados = vuelos[vuelos['TAVG_o'].notnull()]
print 'sin tratar de todos ', vuelosSinTratar.shape
print 'tratados ', vuelosTratados.shape
x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==anyo]
print 'sin tratar totales del año ', x.shape
x = x[x['mesSalida']==mes]
print 'del mes sin tratar ', x.shape
x = x[x['diaSalida']==dia]
print 'vuelos del dia sin tratar ' , x.shape

sin tratar de todos  (168566, 162)
tratados  (44132, 162)
sin tratar totales del año  (22256, 162)
del mes sin tratar  (7592, 162)
vuelos del dia sin tratar  (192, 162)


In [89]:
ddd = vuelosTratados[vuelosTratados['mesSalida']==mes]
ddd = ddd['diaSalida'].unique()
ddd.sort()
ddd

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], dtype=int64)

In [80]:
oo =vuelosSinTratar[vuelosSinTratar['anyoSalida']==2014]

In [74]:
oo = oo['mesSalida'].unique()

In [82]:
oo

,index,airline_code,flight_number,board_point,board_lat,board_lon,board_country_code,off_point,off_lat,off_lon,off_country_code,distance,scheduled_time_of_departure,actual_time_of_departure,scheduled_time_of_arrival,actual_time_of_arrival,departure_delay,arrival_delay,sched_blocktime,est_blocktime,act_blocktime,aircraft_type,aircraft_registration_number,general_status_code,routing,cabin_1_code,cabin_1_fitted_configuration,cabin_1_saleable,cabin_1_pax_boarded,cabin_1_rpk,cabin_1_ask,cabin_2_code,cabin_2_fitted_configuration,cabin_2_saleable,cabin_2_pax_boarded,cabin_2_rpk,cabin_2_ask,total_rpk,total_ask,load_factor,total_pax,total_no_shows,total_cabin_crew,total_technical_crew,total_baggage_weight,number_of_baggage_pieces,mesSalida,anyoSalida,diaSalida,horaSalida,mesLlegada,anyoLlegada,diaLlegada,horaLlegada,diaSemanaSalida,diaSemanaLlegada,TMIN_o,TMIN_d,TMAX_o,TMAX_d,TAVG_o,TAVG_d,SNOW_o,SNOW_d,PRCP_o,PRCP_d,SNWD_o,SNWD_d,ACMC_o,ACMC_d,ACMH_o,ACMH_d,ACSC_o,ACSC_d,ACSH_o,ACSH_d,AWDR_o,AWDR_d,AWND_o,AWND_d,DAEV_o,DAEV_d,DAPR_o,DAPR_d,DASF_o,DASF_d,DATN_o,DATN_d,DATX_o,DATX_d,DAWM_o,DAWM_d,DWPR_o,DWPR_d,EVAP_o,EVAP_d,FMTM_o,FMTM_d,FRGB_o,FRGB_d,FRGT_o,FRGT_d,FRTH_o,FRTH_d,GAHT_o,GAHT_d,MDEV_o,MDEV_d,MDPR_o,MDPR_d,MDSF_o,MDSF_d,MDTN_o,MDTN_d,MDTX_o,MDTX_d,MDWM_o,MDWM_d,MNPN_o,MNPN_d,MXPN_o,MXPN_d,PGTM_o,PGTM_d,PSUN_o,PSUN_d,THIC_o,THIC_d,TOBS_o,TOBS_d,TSUN_o,TSUN_d,WDF1_o,WDF1_d,WDF2_o,WDF2_d,WDF5_o,WDF5_d,WDFG_o,WDFG_d,WDFI_o,WDFI_d,WDFM_o,WDFM_d,WDMV_o,WDMV_d,WESD_o,WESD_d,WESF_o,WESF_d,WSF1_o,WSF1_d,WSF2_o,WSF2_d,WSF5_o,WSF5_d,WSFG_o,WSFG_d,WSFI_o,WSFI_d,WSFM_o,WSFM_d
128062,133158,XX,7863,SKG,40.640280,22.943890,GR,LEU,42.338611,1.409167,ES,1799,2014-12-28 12:15:00,2014-12-28 13:33:00,2014-12-28 12:05:00,2014-12-28 13:10:00,18,5,50,65.0,37,E90,XWSAT,Departed,SKG-LEU,Y,99,92,78,140303,165486,J,1.0,8.0,1.0,1799.0,14390.0,142102,179876,0.790000,79,0,2,2,1399,81,12,2014,28,13:33:00,12,2014,28,13:10:00,domingo,domingo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128414,133517,XX,7163,LEU,42.338611,1.409167,ES,SKG,40.640280,22.943890,GR,1799,2014-12-25 15:00:00,2014-12-25 15:03:00,2014-12-25 20:00:00,2014-12-25 20:01:00,3,1,240,NaN,238,E90,XWZAT,Departed,LEU-SKG,Y,99,96,91,163687,172681,J,1.0,4.0,2.0,3598.0,7195.0,167285,179876,0.930000,93,0,2,2,1533,98,12,2014,25,15:03:00,12,2014,25,20:01:00,jueves,jueves,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128415,133518,XX,7163,LEU,42.338611,1.409167,ES,SKG,40.640280,22.943890,GR,1799,2014-12-26 15:00:00,2014-12-26 15:01:00,2014-12-26 20:00:00,2014-12-26 20:03:00,1,3,240,NaN,242,E90,XWWAT,Departed,LEU-SKG,Y,99,92,92,165486,165486,J,1.0,8.0,8.0,14390.0,14390.0,179876,179876,1.000000,100,0,2,2,1637,98,12,2014,26,15:01:00,12,2014,26,20:03:00,viernes,viernes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128416,133519,XX,7163,LEU,42.338611,1.409167,ES,SKG,40.640280,22.943890,GR,1

In [79]:
vuelosSinTratar['anyoSalida']==2014

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
212651    False
212652    False
212653    False
212654    False
212660    False
212661    False
212662    False
212663    False
212664    False
212665    False
212666    False
212667    False
212668    False
212669    False
212670    False
212671    False
212679    False
212680    False
212681    False
212682    False
212683    False
212684    False
212685    False
212686    False
212689    False
212692    False
212694    False
212695    False
212696    False
212697    False
Name: anyoSalida, dtype:

#############PRUEBAS###########################

In [ ]:
#en este punto debemos pasarle la lat y lon del vuelo

a= closest(np.asarray(lats), [45.800000, 24.150000])
aux = stationsdf[stationsdf['lat']==a[0]]
aux = stationsdf[stationsdf['lng']==a[1]]
stationid = aux['id']
s =stationid.to_string()
stationid = str(s)
stationid = stationid.split()[1]
fecha = tratarFecha('2016-07-18 21:51:00')

In [116]:
del weatherdf['Value3']
del weatherdf['Value4']
del weatherdf['Value5']
#del weatherdf['Value2']

In [ ]:
    u = int(month+day)
    weatherdf = weather2015
    weatherdf = weatherdf[weatherdf['date']-20150000==u]

In [10]:
weatherdf[weatherdf['date']-20150000==107]

,id,date,type,Value1
561698,US1NJGL0001,20150107,PRCP,43
561699,US1NJGL0001,20150107,SNOW,58
561700,US1NJGL0001,20150107,SNWD,58
561701,US1NJGL0001,20150107,WESD,43
561702,US1NJGL0001,20150107,WESF,43
561703,US1NJES0019,20150107,PRCP,15
561704,US1NJES0019,20150107,SNOW,20
561705,US1NJES0019,20150107,WESF,15
561706,ASN00015643,20150107,TMAX,293
561707,ASN00015643,20150107,TMIN,215


In [ ]:
#Now we extract the data from the climate table
datatavg = weatherdf[(weatherdf["id"]=='USW00094860')].sort_values("date")
datatavg["Value1"]=datatavg["Value1"]/10
datatavg = datatavg.reset_index()
datatavg

In [ ]:
 #primero tratamos la fecha
fecha = tratarFecha('20161023')
coor = [45.080689,3.762889]
a= closest(np.asarray(lats), coor)
aux = stationsdf[stationsdf['lat']==a[0]]
aux = stationsdf[stationsdf['lng']==a[1]]
stationid = aux['id']
s =stationid.to_string()
stationid = str(s)
stationid = stationid.split()[1]
    
datatavg = weatherdf[(weatherdf["id"]==stationid)&(weatherdf['date']==fecha)].sort_values("date")
datatavg["Value1"]=datatavg["Value1"]/10
datatavg = datatavg.reset_index()
T = datatavg[datatavg['type']=='PRCP']['Value1']
if T.empty:
    T = 0.0
    return T
else:
    return T

In [112]:
start = time.time()
closest(np.asarray(lats), [45.800000, 24.150000])
end = time.time()
print(end - start)

0.323999881744


In [ ]:
vuelos['actual_time_of_departure']

In [ ]:
var2 = '20160515'
var2[:4]



In [ ]:
x = tratarFecha('2016-05-15 11:41:00')

In [ ]:
str(x)

In [ ]:
vuelos2016.shape

In [ ]:
vuelos2016_01 = vuelos2016[vuelos2016['mesSalida']==1]
vuelos2016_02 = vuelos2016[vuelos2016['mesSalida']==2]


In [ ]:
print vuelos2016_01.shape
print vuelos2016_02.shape

In [ ]:
7*12*31

In [ ]:
def tratar(param,lat, lon, fecha):
    #primero tratamos la fecha
    fecha = tratarFecha(fecha)
    coor = [lat,lon]
    a= closest(np.asarray(lats), coor)
    aux = stationsdf[stationsdf['lat']==a[0]]
    aux = stationsdf[stationsdf['lng']==a[1]]
    stationid = aux['id']
    s =stationid.to_string()
    stationid = str(s)
    stationid = stationid.split()[1]
    datatavg = weatherdf[(weatherdf["id"]==stationid)&(weatherdf['date']==fecha)].sort_values("date")
    datatavg["Value1"]=datatavg["Value1"]/10
    datatavg = datatavg.reset_index()
    T = datatavg[datatavg['type']==param]['Value1']
    if T.empty:
        T = 9999
    return float(T)

In [ ]:
#metodo antiguo
#vuelos2014["TAVG_o"] = vuelos2014.apply(lambda vuelos2014: tratar('TAVG',vuelos2014['board_lat'],vuelos2014['board_lon'],vuelos2014['actual_time_of_departure']), axis = 1)
#vuelos2014["PRCP_o"] = vuelos2014.apply(lambda vuelos2014: tratar('PRCP',vuelos2014['board_lat'],vuelos2014['board_lon'],vuelos2014['actual_time_of_departure']), axis = 1)
#vuelos2014["SNOW_o"] = vuelos2014.apply(lambda vuelos2014: tratar('SNOW',vuelos2014['board_lat'],vuelos2014['board_lon'],vuelos2014['actual_time_of_departure']), axis = 1)
#vuelos2014["TMIN_o"] = vuelos2014.apply(lambda vuelos2014: tratar('TMIN',vuelos2014['board_lat'],vuelos2014['board_lon'],vuelos2014['actual_time_of_departure']), axis = 1)
#vuelos2014["TMAX_o"] = vuelos2014.apply(lambda vuelos2014: tratar('TMAX',vuelos2014['board_lat'],vuelos2014['board_lon'],vuelos2014['actual_time_of_departure']), axis = 1)

In [42]:
mes = 1
dia = 1
mesw = str(mes)
diaw = str(dia)
if dia < 10:
    diaw = '0'+diaw

In [45]:
diaw
 

'01'

In [46]:
mesw

'1'

In [ ]:
weatherdf = weather2015
    weatherdf = weatherdf[weatherdf['date']-20150000==u]